In [4]:
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
from scipy import stats
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics

##load clean data
file_name = 'vehicles.csv'
cleaner = pd.read_csv('vehicles.csv')
cleaner = cleaner[["price", "year", "manufacturer", "model", "condition", "cylinders", "fuel", "odometer", "drive"]]

In [5]:
cleaner.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,drive
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cleaner.sample(40)

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,drive
127602,28988,2017.0,infiniti,qx50,NaN,6 cylinders,gas,31132.0,rwd
18362,0,2006.0,ford,f550,NaN,NaN,diesel,0.0,NaN
310589,4995,2006.0,honda,civic,NaN,NaN,gas,173558.0,NaN
406388,41999,2017.0,ram,1500,NaN,8 cylinders,gas,62834.0,4wd
187503,9900,2007.0,chevrolet,tahoe lt 4wd,excellent,NaN,gas,126000.0,4wd
190458,3795,2011.0,honda,civic,excellent,4 cylinders,gas,249800.0,fwd
318488,6995,2013.0,nissan,versa,NaN,NaN,gas,109531.0,NaN
184736,41990,2019.0,lincoln,continental reserve,good,6 cylinders,gas,9345.0,fwd
292549,23989,2018.0,gmc,terrain,NaN,4 cylinders,gas,24194.0,4wd
180179,5500,2004.0,ford,expedition eddie bauer,NaN,NaN,gas,143000.0,NaN


In [7]:
## Clean and fill

In [10]:
#print(list(cleaner))
cleaner = cleaner[(cleaner.price < 70000) & (cleaner.price >= 800)]
cleaner = cleaner[(cleaner.odometer < 130000)]
#print(cleaner.shape)
cleaner.loc[cleaner.year >= 2021, 'condition'] = cleaner.loc[cleaner.year >= 2021, 'condition'].fillna('new')
cleaner.loc[cleaner.year >= 2019, 'condition'] = cleaner.loc[cleaner.year >= 2019, 'condition'].fillna('like new')
#print(cleaner.condition.value_counts())


##odometer reading
excellent_odo_mean = cleaner[cleaner['condition'] == 'excellent']['odometer'].mean()
good_odo_mean = cleaner[cleaner['condition'] == 'good']['odometer'].mean()
like_new_odo_mean = cleaner[cleaner['condition'] == 'like new']['odometer'].mean()
salvage_odo_mean = cleaner[cleaner['condition'] == 'salvage']['odometer'].mean()
fair_odo_mean = cleaner[cleaner['condition'] == 'fair']['odometer'].mean()

cleaner.loc[cleaner['odometer'] <= like_new_odo_mean, 'condition'] = cleaner.loc[cleaner['odometer'] <= like_new_odo_mean, 'condition'].fillna('like new')
cleaner.loc[cleaner['odometer'] >= fair_odo_mean, 'condition'] = cleaner.loc[cleaner['odometer'] >= fair_odo_mean, 'condition'].fillna('fair')
cleaner.loc[((cleaner['odometer'] > like_new_odo_mean) &
       (cleaner['odometer'] <= excellent_odo_mean)), 'condition'] = cleaner.loc[((cleaner['odometer'] > like_new_odo_mean) &
       (cleaner['odometer'] <= excellent_odo_mean)), 'condition'].fillna('excellent')

cleaner.loc[((cleaner['odometer'] > excellent_odo_mean) &
       (cleaner['odometer'] <= good_odo_mean)), 'condition'] = cleaner.loc[((cleaner['odometer'] > excellent_odo_mean) &
       (cleaner['odometer'] <= good_odo_mean)), 'condition'].fillna('good')

cleaner.loc[((cleaner['odometer'] > good_odo_mean) &
       (cleaner['odometer'] <= fair_odo_mean)), 'condition'] = cleaner.loc[((cleaner['odometer'] > good_odo_mean) &
       (cleaner['odometer'] <= fair_odo_mean)), 'condition'].fillna('salvage')

cleaner['cylinders'] = cleaner['cylinders'].fillna(method='ffill')
cleaner['fuel'] = cleaner['fuel'].fillna(method='ffill')
cleaner['drive'] = cleaner['drive'].fillna(method='ffill')
cleaner['manufacturer'] = cleaner['manufacturer'].fillna(method='ffill')
cleaner['year'] = cleaner['year'].fillna(method='ffill')
cleaner['model'] = cleaner['model'].fillna(method='ffill')



In [12]:
cleaner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270823 entries, 27 to 426879
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         270823 non-null  int64  
 1   year          270823 non-null  float64
 2   manufacturer  270823 non-null  object 
 3   model         270823 non-null  object 
 4   condition     270823 non-null  object 
 5   cylinders     270823 non-null  object 
 6   fuel          270823 non-null  object 
 7   odometer      270823 non-null  float64
 8   drive         270819 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 20.7+ MB


In [13]:
cleaner['drive'].isna().sum()

4

In [14]:
cleaner['drive'][0:10]

27    NaN
28    NaN
29    NaN
30    NaN
31    rwd
32    4wd
33    4wd
34    4wd
35    4wd
36    rwd
Name: drive, dtype: object

In [21]:
cleaner['drive'][0:10].fillna(method='ffill')

27    NaN
28    NaN
29    NaN
30    NaN
31    rwd
32    4wd
33    4wd
34    4wd
35    4wd
36    rwd
Name: drive, dtype: object

In [23]:
missing_drive = cleaner['drive'].isna()

In [25]:
cleaner.loc[missing_drive, 'drive'] = 'fwd'

In [26]:
cleaner['drive'][0:10]

27    fwd
28    fwd
29    fwd
30    fwd
31    rwd
32    4wd
33    4wd
34    4wd
35    4wd
36    rwd
Name: drive, dtype: object

In [27]:
cleaner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270823 entries, 27 to 426879
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         270823 non-null  int64  
 1   year          270823 non-null  float64
 2   manufacturer  270823 non-null  object 
 3   model         270823 non-null  object 
 4   condition     270823 non-null  object 
 5   cylinders     270823 non-null  object 
 6   fuel          270823 non-null  object 
 7   odometer      270823 non-null  float64
 8   drive         270823 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 20.7+ MB


In [29]:
cleaner.to_csv('clean_data.csv', index = False)

In [31]:
cleaner.isnull().sum()

price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
drive           0
dtype: int64